This notebook walks you through the procedure to generate cropped/downsampled/correctly aligned blocks of the ground truth and predicted segmentation using the SegPrep module. You can also filter the predicted segmentation for fibers. Once the segmentation blocks are generated, you can perform an error analysis using voxel-based methods (IoU, VI) and skeleton-based methods.

# SegPrep

In [1]:
import json

from cerebellum.data_prep.seg_prep import *

# json file with data locations and global dataset params
# a sample is included in the package, but you need to populate it with relvant file locations and params
with open('data_locs.json') as f:
	data_locs = json.load(f)

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Ground truth

In [2]:
print "Preparing GT"
resolution = (30, 48, 48)
gt48nm_file = data_locs["gt"]["dir"] + data_locs["gt"]["48nm"] # path to ground truth
bbox = data_locs["gt"]["48nm-bbox"] # global bbox co-ordinates
i = 1 # TO CHANGE: block index
print "Generating block %d from"%(i)
print gt48nm_file
zz = i*data_locs["block-size"]+data_locs["aff-offset"]
gt_block_name = "gt%04d"%(zz)
gt_block = SegPrep(gt_block_name, resolution)
block_lims = ((data_locs["block-size"]*i,data_locs["block-size"]*(i+1)),
              (bbox[1],bbox[4]),(bbox[2],bbox[5]))
print block_lims
gt_block.read(gt48nm_file, "main", block_lims=block_lims)
print gt_block.shape
gt_block.write()
#gt_block.gen_bboxes() # NOTE: this is the longest step
gt_block.read_bboxes() # Assuming bboxes are generated, use this to read them in
gt_block.relabel(use_bboxes=True, print_labels=False) # get ordered labels - essential for skeletonization
gt_block.write() # this saves your segmentation to the ./segs/<gt_block_name>/ folder

Preparing GT
Generating block 1 from
/home/srujanm/cerebellum/data/gt48nm_pf-align1_1k.h5
((90, 180), (16, 556), (208, 696))
[90, 540, 488]
Starting relabeling of 1006 objects
Relabeling time: 0.381241


In [4]:
# skeletonize GT for error analysis
from cerebellum.skeletonize import gen_skeletons
gen_skeletons(gt_block_name, resolution, overwrite_prev=True)

Starting skeletonization of gt0104
Downsampling to resolution (80, 80, 80) in 0.506502866745 seconds
Topological thinning time for (80, 80, 80): 7.18329906464
Endpoint vector time for (80, 80, 80): 1.64452314377
Edge finding time for (80, 80, 80): 0.331279993057


## Predicted segmentation

In [5]:
print "Preparing pred"
i = 1 # TO CHANGE: block index
dsmpl = (1,6,6)
resolution = (30,48,48)
offset = data_locs["initial-seg"]["8nm-offset"] # offset aligns segmentation with GT
print "Generating block %d from"%(i)
seg_file = data_locs["initial-seg"]["dir"]+data_locs["initial-seg"]["8nm-all"]
zz = i*data_locs["block-size"]+data_locs["aff-offset"]
if zz!=data_locs["aff-offset"]: # adjust block index
    seg_file = seg_file[:-7]+"%04d.h5"%(zz)
print seg_file
seg_block_name = "pred-48nm-crop2gt-%04d"%(zz)
seg_block = SegPrep(seg_block_name, resolution)
bbox = data_locs["gt"]["48nm-bbox"]
seg_block_lims = ((0,data_locs["block-size"]),
                  (max(0,dsmpl[1]*bbox[1]+offset[1]),dsmpl[1]*bbox[4]+offset[1]),
                  (max(0,dsmpl[2]*bbox[2]+offset[2]),dsmpl[2]*bbox[5]+offset[2]))
print seg_block_lims
seg_block.read(seg_file, "main", dsmpl=dsmpl, block_lims=seg_block_lims)
print seg_block.shape
seg_block.padzeros(7, 1) # manually chosen to match GT block size, can easily insert a variable here
print seg_block.shape
seg_block.write()  # this saves your segmentation to the ./segs/<seg_block_name>/ folder

Preparing pred
Generating block 1 from
/mnt/hp03/donglai/public/cere_pf/data_max_label/0104.h5
((0, 90), (52, 3292), (1204, 4132))
[90, 533, 488]
(90, 540, 488)


## Fiber extraction

In this stage, you zero out parts of the segmentation that are not fibers. See the `find_fiber_ids()` function in the `SegPrep` module for various filter methods.

In [8]:
# seg_block.gen_bboxes() # generate bboxes of objects in segmentaiton
# If boxes are generated, proceed with below steps
seg_block.read_bboxes()

# filter params
filter_method = "dsmpl"
filter_params = {"dsmpl": (4,3,3),
                 "bvol-thresh": float(data_locs["block-size"])/4*40}

seg_block.find_fiber_ids(method=filter_method, params=filter_params) # see function for more details on setting non-default filter method and params
seg_block.filter_fibers() # see function for more details on setting non-default filter method and params
#seg_block.relabel(use_bboxes=True) # relabel if you wish
seg_block.write(stage="filt-"+filter_method) # this saves your filtered segmentation to the ./segs/<seg_block_name>/ folder

Found 62 cell bodies
Fiber extraction time: 0.153320 s
Zeroing 62 objects that are not fibers
Fiber filtering time: 1.591535


# Error analysis

First run error analysis on unfiltered segmentation.

In [9]:
from cerebellum.error_analysis.skel_segeval import *

# voxel methods - short evaluation
# results are saved to ./err-analysis/<seg_block_name>/folder
vox_eval = VoxEval(gt_block_name, seg_block_name, stage=None)
vox_eval.find_misses() # find missing objects
vox_eval.find_vi()

Starting voxel evaluation methods for pred-48nm-crop2gt-0104 against gt0104
Found 8 GT objects completely missing
VI split, VI merge: 0.229444, 0.219173


In [10]:
# voxel methods - long evaluations
# run this to generate a histogram of objects in order of deltaVI_split and deltaVI_merge
# results are saved to ./err-analysis/<seg_block_name>/folder

# vox_eval.run_fullsuite(iou_max=0.6, hist_segs=10, overwrite_prev=False)

In [11]:
# set skeleton error analysis thresholds - see the SkelEval module for details
t_om = 0.9 # omission threshold
t_m = 0.5 # merge threshold
t_s = 0.8 # split threshold

In [12]:
# skeleton methods
# evaluate unfiltered pred against GT skeletons
# results are saved to ./err-analysis/<seg_block_name>/folder
skel_eval = SkelEval(gt_block_name, seg_block_name, dsmpl_res=(80,80,80), 
                     t_om=t_om, t_m=t_m, t_s=t_s, 
                     include_zero_split=False, include_zero_merge=True,
                     stage=None, overwrite_prev=True)

Starting error analysis of pred-48nm-crop2gt-0104 against skeletons of gt0104
Warning! Labels in predicted segmentation are not consecutive. Proceeding anyway
Starting evaluation of 29921 labels in 90x540x488 predicted segmentation against 1006 GT skeletons
Using error thresholds: t_om=0.90, t_m=0.50, t_s=0.80
Skeleton evaluation time: 5.21822786331
Results:
1 omissions, 86 merges, 307 splits, 54 hybrid, 558 correct
GT ERL: 4899, Prediction ERL: 3040
GT TRL: 3671311, Prediction TRL: 2633857
Omitted RL: 0, Merged RL: 512808, Split RL: 524645


In [13]:
# validate skeleton methods
# apply oracle to IDs flagged as splits and merges and check change in VI
skel_eval.merge_oracle()
skel_eval.split_oracle()

Starting voxel evaluation methods for pred-48nm-crop2gt-0104 against gt0104
Original VI split, VI merge: 0.229444, 0.219173
After fixing 140 merges in GT flagged by skeleton analysis:
(0.19892408344535406, 0.09704647751885559)
Starting voxel evaluation methods for pred-48nm-crop2gt-0104 against gt0104
Original VI split, VI merge: 0.229444, 0.219173
After fixing 361 splits in GT flagged by skeleton analysis:
(0.07890341036428314, 0.06783597265097327)


Now carry error analysis on filtered segmentation. The error statistics should be as close as possible to the unfiltered segmentation, if your filtering method is doing a good job. You can go back up to the filtering step and readjust the filter params to improve this result

In [15]:
# repeat error analysis for filtered objects
# results are saved to ./err-analysis/<seg_block_name>/<filtered>
vox_eval = VoxEval(gt_block_name, seg_block_name, stage='filtered')
vox_eval.find_misses() # find missing objects
vox_eval.find_vi()
skel_eval = SkelEval(gt_block_name, seg_block_name, dsmpl_res=(80,80,80), 
                     t_om=0.8, t_m=0.2, t_s=0.7, 
                     stage='filtered', overwrite_prev=True)

Starting voxel evaluation methods for pred-48nm-crop2gt-0104 against gt0104
Found 51 GT objects completely missing
VI split, VI merge: 0.209012, 0.270250
Starting error analysis of pred-48nm-crop2gt-0104 against skeletons of gt0104
Starting evaluation of 5070 labels in 90x540x488 predicted segmentation against 1006 GT skeletons
Using error thresholds: t_om=0.80, t_m=0.20, t_s=0.70
Skeleton evaluation time: 4.98642706871
Results:
19 omissions, 119 merges, 106 splits, 57 hybrid, 705 correct
GT ERL: 4899, Prediction ERL: 3197
GT TRL: 3671311, Prediction TRL: 2648101
Omitted RL: 56026, Merged RL: 705352, Split RL: 261830
